This Jupyter Notebook serves as a tool to convert CSV files to LAS files. It utilizes the lasio library to facilitate the conversion process. The notebook provides an interactive interface where users can select the CSV file they want to convert and choose the destination path and name for the resulting LAS file. By running the notebook and following the prompts, users can easily transform their CSV data into the LAS format commonly used in the oil and gas industry for storing geophysical log data. The conversion process ensures seamless compatibility and enables users to leverage LAS file features for data analysis and interpretation

## Libraries
Libraries to be used during code development are imported.

In [3]:
import lasio
import pandas as pd 
import tkinter as tk
from tkinter import filedialog
from tkinter.filedialog import askopenfilename

## Import Database 
In this section of the script the data to be plotted is imported. It is required that the file to work with is in a comma separated text format ".csv". Then a summary table with database statistics is generated.

In [30]:
# The root window is created.
root = tk.Tk()
# show askopenfilename dialog without the Tkinter window
root.withdraw()

# select csv file
datos = askopenfilename(filetypes=[("Csv files", "*.csv")])

## read the data and do rank transformation
#rgp= pd.read_csv(datos, index_col=0)
rgp= pd.read_csv(datos)
rgp.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,DEPT,ECGR,AT90,RHOZ,DTCO,DTSM,NPHI
count,7513.00000,7513.00000,7513.00000,7513.00000,7513.00000,7513.00000,7513.00000
mean,1278.33120,68.85613,3.97952,2.51642,90.30831,188.94403,0.26773
std,330.54960,13.25824,3.23325,0.07318,12.16394,36.78915,0.06462
min,705.91680,22.97560,0.57430,2.18300,59.41090,112.59790,0.06510
25%,992.12400,62.39860,2.19330,2.44130,81.97110,165.42280,0.22840
50%,1278.33120,70.09090,3.33640,2.53820,88.36340,179.16620,0.26180
75%,1564.53840,78.27730,4.22640,2.57260,102.83740,221.32770,0.32100
max,1850.74560,167.69470,60.65310,2.72670,121.13490,311.19950,0.49110


In [31]:
las_file = lasio.LASFile()

## Parameters
Add the well parameters as well name, field name, UWI, define NULL number and Kelly Bushing

In [32]:
well_name = 'Well 1' #well name
field_name = 'Random Field'
uwi = '123456'
null = -999.25
ekb = 151.86

In [33]:
las_file.well['WELL'] = lasio.HeaderItem('WELL', value = well_name)
las_file.well['FLD'] = lasio.HeaderItem('FLD', value = field_name)
las_file.well['UWI'] = lasio.HeaderItem('UWI', value = uwi)
las_file.well['NULL'] = lasio.HeaderItem('NULL', value = null)
las_file.params['EKB'] = lasio.HeaderItem('EKB', value=ekb)

### Add the depth curve
Choose m or ft

In [ ]:
las_file.add_curve('DEPT', rgp['DEPT'], unit = 'm')

### LAS file header 

In [35]:
las_file.header

{'Version': [HeaderItem(mnemonic="VERS", unit="", value="2.0", descr="CWLS log ASCII Standa"),
  HeaderItem(mnemonic="WRAP", unit="", value="NO", descr="One line per depth ste"),
  HeaderItem(mnemonic="DLM", unit="", value="SPACE", descr="Column Data Section ")],
 'Well': [HeaderItem(mnemonic="STRT", unit="m", value="nan", descr="START DEPTH"),
  HeaderItem(mnemonic="STOP", unit="m", value="nan", descr="STOP DEPTH"),
  HeaderItem(mnemonic="STEP", unit="m", value="nan", descr="STEP"),
  HeaderItem(mnemonic="NULL", unit="", value="-999.25", descr=""),
  HeaderItem(mnemonic="COMP", unit="", value="", descr="COMPANY"),
  HeaderItem(mnemonic="WELL", unit="", value="AGF0705", descr=""),
  HeaderItem(mnemonic="FLD", unit="", value="Random Field", descr=""),
  HeaderItem(mnemonic="LOC", unit="", value="", descr="LOCATION"),
  HeaderItem(mnemonic="PROV", unit="", value="", descr="PROVINCE"),
  HeaderItem(mnemonic="CNTY", unit="", value="", descr="COUNTY"),
  HeaderItem(mnemonic="STAT", unit="",

## Add the units for each curve

In [36]:
units = ['m', 'API', 'ohm.m', 'g/cm3', 'us/ft', 'us/ft', 'v/v_decimal'] #Well 1

In [37]:
for col, unit in zip(rgp.columns, units):
    if col != 'DEPT':
        las_file.add_curve(col, rgp[col], unit = unit)

In [38]:
las_file.curves

[CurveItem(mnemonic="DEPT", unit="m", value="", descr="", original_mnemonic="DEPT", data.shape=(7513,)),
 CurveItem(mnemonic="ECGR", unit="API", value="", descr="", original_mnemonic="ECGR", data.shape=(7513,)),
 CurveItem(mnemonic="AT90", unit="ohm.m", value="", descr="", original_mnemonic="AT90", data.shape=(7513,)),
 CurveItem(mnemonic="RHOZ", unit="g/cm3", value="", descr="", original_mnemonic="RHOZ", data.shape=(7513,)),
 CurveItem(mnemonic="DTCO", unit="us/ft", value="", descr="", original_mnemonic="DTCO", data.shape=(7513,)),
 CurveItem(mnemonic="DTSM", unit="us/ft", value="", descr="", original_mnemonic="DTSM", data.shape=(7513,)),
 CurveItem(mnemonic="NPHI", unit="v/v_decimal", value="", descr="", original_mnemonic="NPHI", data.shape=(7513,))]

## Save your LAS file

In [5]:
def guardar_archivo():
    # Prompt user to select the file path and name
    file_path = filedialog.asksaveasfilename(defaultextension=".las")
    if file_path:
        # Write the file using the selected file path
        las_file.write(file_path)

# Create the main window
window = tk.Tk()

# Add a button to save the file
save_button = tk.Button(window, text="Save file", command=guardar_archivo)
save_button.pack()

# Start the event loop
window.mainloop()